<a href="https://colab.research.google.com/github/vanesalinas/HenryPI_MLOps/blob/main/funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [2]:
try:
    df_steam_games = pd.read_json('./Datasets/clean_steam_games.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})

In [3]:
# Convierte la columna 'release_date' a tipo datetime
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')

# Encuentra los registros que no pudieron convertirse correctamente a formato de fecha
registros_sin_fecha = df_steam_games[df_steam_games['release_date'].isnull()]

if registros_sin_fecha.empty:
    print("Todos los registros tienen formato de fecha en la columna 'release_date'.")
else:
    print("Los siguientes registros no tienen formato de fecha en la columna 'release_date':")
    print(registros_sin_fecha)

Los siguientes registros no tienen formato de fecha en la columna 'release_date':
                                                  genres           app_name  \
9        'Action', 'Indie', 'Simulation', 'Early Access'    Uncanny Islands   
34                                            'Strategy'     United Tactics   
35     'Indie', 'Massively Multiplayer', 'RPG', 'Earl...       Life of Rome   
36     'Adventure', 'Free to Play', 'RPG', 'Early Acc...     Idle Adventure   
235                                 'Adventure', 'Indie'   Blueberry Garden   
...                                                  ...                ...   
28635                        'Action', 'Casual', 'Indie'           BOMBFEST   
28637          'Action', 'Casual', 'Indie', 'Simulation'   Lonely Astronaut   
28638     'Action', 'Adventure', 'Indie', 'Early Access'  RECHARGE COMPLETE   
28647                                  'Action', 'Indie'         Tetrapulse   
28669  'Casual', 'Indie', 'Simulation', 'Sports',

In [4]:
# Elimina los registros con NaT en la columna 'release_date'
df_steam_games = df_steam_games.dropna(subset=['release_date'])

In [5]:
try:
    df_user_reviews = pd.read_json('./Datasets/clean_user_reviews.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

In [6]:
try:
    df_user_items = pd.read_json('./Datasets/clean_user_items.json', lines=True)
except ValueError as e:
    print(f"Error al leer el archivo JSON: {e}")

In [36]:
def developer( desarrollador : str ):
    #Transformar la columna "developer" y "price" en str en letra minuscula
    df_steam_games['developer'] = df_steam_games['developer'].astype(str).str.lower()
    df_steam_games['price'] = df_steam_games['price'].astype(str).str.lower()
    desarrollador.lower()

    #Filtrar por desarrollador
    developer_df = df_steam_games[df_steam_games['developer'].str.contains(desarrollador, case=False)]

    if developer_df.empty:
        print(f"No hay registros para el desarrollador {desarrollador}.")
        return None

    estadisticas_desarrolladores = []

    for developer, juegos in developer_df.groupby('developer'):
        total_juegos = len(juegos)
        juegos_free = juegos[juegos['price'].str.contains('free', case=False)]
        total_juegos_free = len(juegos_free)

        porcentaje_free = (total_juegos_free / total_juegos) * 100 if total_juegos > 0 else 0

        estadisticas = {
            'desarrollador': developer,
            'total_juegos': total_juegos,
            'porcentaje_free': porcentaje_free
        }

        estadisticas_desarrolladores.append(estadisticas)

    return estadisticas_desarrolladores

In [37]:
developer('nikita')

[{'desarrollador': 'korovkin nikita,filatov igor,lagunov roman,kristian shatov',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita', 'total_juegos': 2, 'porcentaje_free': 0.0},
 {'desarrollador': 'nikita "ghost_rus"',
  'total_juegos': 9,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita "ghost_rus",underanger thegachiwarrior',
  'total_juegos': 2,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita kaf', 'total_juegos': 2, 'porcentaje_free': 50.0},
 {'desarrollador': 'nikita markin,artem basko',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita nefedov',
  'total_juegos': 1,
  'porcentaje_free': 0.0},
 {'desarrollador': 'nikita online',
  'total_juegos': 4,
  'porcentaje_free': 75.0}]

In [17]:
def userdata( User_id : str ):
    '''
    Calcula cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items

    Parametros
    ----------
    User_id : str

    Retorno
    -------
    {"Usuario X" : nombre de usuario, "Dinero gastado": x USD, "% de recomendación": x%, "cantidad de items": x}

    '''

    # Unir los dataframes usando 'item_id' como clave
    df_merged = pd.merge(df_steam_games, df_user_reviews, on='item_id')

    # Filtrar juegos comprados por el usuario
    juegos_comprados = df_merged[df_merged['user_id'] == User_id].copy()

    # Convertir la columna 'price' a tipo cadena
    juegos_comprados['price'] = juegos_comprados['price'].astype(str)
    # Reemplazar "free" o "Free" en la columna 'price' por 0
    juegos_comprados.loc[juegos_comprados['price'].str.contains(r'(?i)\bfree\b'), 'price'] = 0
    # Convertir la columna 'price' a tipo float para poder sumar los valores
    juegos_comprados['price'] = juegos_comprados['price'].astype(float)
    
    # Convertir la columna 'price' a tipo numérico
    juegos_comprados['price'] = pd.to_numeric(juegos_comprados['price'], errors='coerce')

    # Calcular dinero gastado por el usuario
    dinero_gastado = round(juegos_comprados['price'].sum(), 2)

    # Calcular el porcentaje de recomendación
    total_reviews = juegos_comprados.shape[0]
    reviews_recomendadas = juegos_comprados[juegos_comprados['recommend'] == True].shape[0]
    porcentaje_recomendacion = (reviews_recomendadas / total_reviews) * 100 if total_reviews > 0 else 0

    # Calcular cantidad de items
    cantidad_items = juegos_comprados.shape[0]

    return {
        "Usuario": User_id,
        "Dinero gastado": f"{dinero_gastado} USD",
        "% de recomendación": f"{porcentaje_recomendacion}%",
        "Cantidad de items": cantidad_items
    }

In [18]:
userdata('js41637')

{'Usuario': 'js41637',
 'Dinero gastado': '29.98 USD',
 '% de recomendación': '100.0%',
 'Cantidad de items': 2}

In [49]:
def UserForGenre( genero : str ):
    '''
    Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

    Parametros
    ----------
    genero : str

    Retorno
    -------
    {"Usuario con más horas jugadas para Género X" : nombre de usuario, "Horas jugadas":[{Año: x1, Horas: n}, {Año: x2, Horas: n2}, {Año: x3, Horas: n3}]}

    '''
    genero.lower()
    df_steam_games['genres'] = df_steam_games['genres'].astype(str).str.lower()

    # Filtrar el DataFrame df_steam_games por el género dado
    df_filtered = df_steam_games[df_steam_games['genres'].str.contains(genero, na=False)].copy()
        
    # Extraer el año de lanzamiento
    df_filtered['release_year'] = df_filtered['release_date'].dt.year
        
    # Combinar los DataFrames utilizando 'item_id'
    df_combined = pd.merge(df_user_items, df_filtered, on='item_id', how='inner')
        
    # Obtener el usuario con más horas jugadas para el género dado
    usuario_max_horas = df_combined.groupby('user_id')['playtime_forever'].sum().idxmax()
        
    # Obtener las horas jugadas por el usuario con más horas jugadas\n",
    horas_usuario_max = df_combined[df_combined['user_id'] == usuario_max_horas]['playtime_forever'].sum()
        
    #renombrar columnas
    df_combined = df_combined.rename(columns={'release_year': 'Año', 'playtime_forever': 'Horas Jugadas'})
        
    # Agrupar por año de lanzamiento y sumar las horas jugadas\n",
    horas_por_año = df_combined.groupby('Año')['Horas Jugadas'].sum().reset_index()
        
    return {
        "Usuario con más horas jugadas para " + genero: usuario_max_horas,
        "Horas jugadas": horas_por_año.to_dict(orient='records'),
        "Horas totales jugadas por el usuario": horas_usuario_max
    }

In [50]:
UserForGenre('adventure')

{'Usuario con más horas jugadas para adventure': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 1983, 'Horas Jugadas': 3473},
  {'Año': 1984, 'Horas Jugadas': 384},
  {'Año': 1987, 'Horas Jugadas': 7712},
  {'Año': 1988, 'Horas Jugadas': 29963},
  {'Año': 1989, 'Horas Jugadas': 9311},
  {'Año': 1990, 'Horas Jugadas': 5492},
  {'Año': 1991, 'Horas Jugadas': 1722},
  {'Año': 1992, 'Horas Jugadas': 23795},
  {'Año': 1993, 'Horas Jugadas': 46772},
  {'Año': 1994, 'Horas Jugadas': 15890},
  {'Año': 1995, 'Horas Jugadas': 10026},
  {'Año': 1996, 'Horas Jugadas': 104199},
  {'Año': 1997, 'Horas Jugadas': 324405},
  {'Año': 1998, 'Horas Jugadas': 176987},
  {'Año': 1999, 'Horas Jugadas': 7735},
  {'Año': 2000, 'Horas Jugadas': 105121},
  {'Año': 2001, 'Horas Jugadas': 50619},
  {'Año': 2002, 'Horas Jugadas': 184447},
  {'Año': 2003, 'Horas Jugadas': 5189429},
  {'Año': 2004, 'Horas Jugadas': 96149},
  {'Año': 2005, 'Horas Jugadas': 206433},
  {'Año': 2006, 'Horas Jugadas': 203976},
  {'Año': 2007,

In [62]:
def best_developer_year( año : int ):
    '''
    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

    Parametros
    ----------
    año : int

    Retorno
    -------
    [{"Puesto x1" : X}, {"Puesto x2" : Y},{"Puesto x3" : Z}]

    '''
    # Unir los DataFrames en función de la columna 'item_id'
    df_completo = pd.merge(df_steam_games, df_user_reviews, on='item_id', how='inner')

    # Filtrar los juegos por el año especificado
    juegos_del_año = df_completo[df_completo['release_date'].dt.year == año]

    # Filtrar los juegos recomendados y los comentarios positivos
    juegos_recomendados = juegos_del_año[juegos_del_año['recommend'] == True]
    comentarios_positivos = juegos_del_año[juegos_del_año['sentiment_analysis'] == 2]

    # Calcular la suma de recomendaciones y comentarios positivos por desarrollador
    recomendaciones_por_desarrollador = juegos_recomendados.groupby('developer').size()
    comentarios_por_desarrollador = comentarios_positivos.groupby('developer').size()

    # Calcular la puntuación total por desarrollador sumando recomendaciones y comentarios positivos
    puntuacion_por_desarrollador = recomendaciones_por_desarrollador.add(comentarios_por_desarrollador, fill_value=0)

    # Seleccionar los tres primeros desarrolladores con la puntuación más alta
    top_3_desarrolladores = puntuacion_por_desarrollador.nlargest(3)

    # Crear la lista de retorno en el formato especificado
    retorno = [{"Puesto {}: {}".format(i+1, desarrollador): puntuacion} for i, (desarrollador, puntuacion) in enumerate(top_3_desarrolladores.items())]

    if top_3_desarrolladores.empty:
        print(f"No hay registros para el año {año}.")
        return None
    
    return retorno


In [63]:
best_developer_year(2010)

[{'Puesto 1: firaxis games,aspyr (mac, linux)': 546.0},
 {'Puesto 2: taleworlds entertainment': 339.0},
 {'Puesto 3: obsidian entertainment': 287.0}]

In [52]:
def developer_reviews_analysis( desarrolladora : str ):
    '''
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

    Parametros
    ----------
    desarrolladora : str

    Retorno
    -------
    {'desarrollador x' : [Negative = x1, Positive = x2]}

    '''
    desarrolladora.lower()
    df_steam_games['developer'] = df_steam_games['developer'].astype(str).str.lower()

    # Fusionar los DataFrames por item_id
    merged_df = pd.merge(df_steam_games, df_user_reviews, how='inner', on='item_id')

    # Filtrar por el desarrollador buscado
    desarrolladora_df = merged_df[merged_df['developer'].str.contains(desarrolladora, case=False)]

    if desarrolladora_df.empty:
        print(f"No hay registros para el desarrollador {desarrolladora}.")
        return None

    #contar la cantidad de reseñas
    for developer in desarrolladora_df.groupby('developer'):
        cantidad_positivas = (desarrolladora_df['sentiment_analysis'] == 2).sum()
        cantidad_negativas = (desarrolladora_df['sentiment_analysis'] == 0).sum()

        resultado = {
            'positivas': cantidad_positivas,
            'negativas': cantidad_negativas
        }

    return {desarrolladora: [f"{key} = {value}" for key, value in resultado.items()]}

In [53]:
developer_reviews_analysis('nikita')

{'nikita': ['positivas = 3', 'negativas = 0']}